The contents of this course including lectures, labs, homework assignments, and exams have all been adapted from the [Data 8 course at University California Berkley](https://data.berkeley.edu/education/courses/data-8). Through their generosity and passion for undergraduate education, the Data 8 community at Berkley has opened their content and expertise for other universities to adapt in the name of undergraduate education.

In [ ]:
!pip install datascience -q
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

# Chapter 7 Review

#### What type of data do you typically use with line graphs?
<font color='blue'>Double click then replace this sentence with your answer so it will be blue.</font>

#### What type of data do you typically use with scatter plots?
<font color='blue'>Double click then replace this sentence with your answer so it will be blue.</font>

#### What type of data do you typically use with bar charts?
<font color='blue'>Double click then replace this sentence with your answer so it will be blue.</font>

#### What type of data do you typically use with histogram plots?
<font color='blue'>Double click then replace this sentence with your answer so it will be blue.</font>

In [ ]:
# Import the skyscrappers.csv as skyscrapers


In [ ]:
# Plot completed vs height


In [ ]:
# Plot the height of the sky scrapers. Make another plot of when they were completed.


In [ ]:
#Using the histogram plot, calculate how many skyscrapers are between 250 and 300 meters?
# Hint: make the histogram bins a useful width


In [ ]:
# Verify your above answer using data from the skyscraper table.


In [ ]:
# Using the histogram plot, calculate how many skyscrapers were completed between 2010 and 2020?
# Hint: make the histogram bins a useful width


In [ ]:
# Verify your above answer using data from the skyscraper table.


In [ ]:
# BONUS:
# Plot the average height by composite


In [ ]:
# BONUS:
# Plot the average height by year

# Chapter 8: Functions and Tables

In [ ]:
def triple(x):
    '''triple x'''
    return 3*x
triple(10)

### Function synatx

<img src=Function.PNG style="width: 900px;"/>

### Note About Scopes

In [ ]:
x

In [ ]:
x = 5

In [ ]:
triple(2 * x)

In [ ]:
x

### Type Agnostic

In [ ]:
triple('ha')

In [ ]:
triple(np.arange(4))

## What does the following function do?  What kind of input does it take?

In [ ]:
def percent_of_total(s):
    return np.round(s / sum(s) * 100, 2)

In [ ]:
percent_of_total(make_array(1,2,3,4,5))

In [ ]:
percent_of_total(make_array(1, 213, 38))

### Multiple Arguments

$ c^2 = a^2 + b^2 \hspace{20 pt} => \hspace{20 pt} c = \sqrt{ a^2 + b^2 } $

In [ ]:
def hypotenuse(a,b):
    hypot_squared = (a ** 2 + b ** 2)
    return hypot_squared ** 0.5

In [ ]:
hypotenuse(9, 12)

In [ ]:
hypotenuse(3, 4)

## Applying functions to Columns
The apply method creates an array by applying a function on every element in input column(s)
* First argument: Function to apply
* Other arguments: The input column(s)

**table_name.apply(function_name, 'column_label')**

In [ ]:
ages = Table().with_columns(
    'Person', make_array('Jim', 'Pam', 'Michael', 'Creed'),
    'Birth Year', make_array(1985, 1988, 1967, 1904)
)
ages

In [ ]:
def cap_at_1980(x):
    return min(x, 1980)

In [ ]:
cap_at_1980(1975)

In [ ]:
cap_at_1980(1991)

In [ ]:
ages.apply(cap_at_1980, 'Birth Year')

In [ ]:
def pick_up_phone(name):
    return 'Dunder Mifflin, this is ' + name

In [ ]:
ages.apply(pick_up_phone, 'Person')

In [ ]:
def name_and_age(name, year):
    age = 2021 - year
    return name + ' is ' + str(age)

In [ ]:
ages.apply(name_and_age, 'Person', 'Birth Year')

## Prediction
We will use a data set collected by Sir Francis Galton (1822 - 1911).  The table has the heights of parents and their adult children (in inches).  The "midparentHeight" column is the acerage of height of the two parents. The "childNum is the childs birth rank (1 = oldest).  The dataset was collected to predict child height from parents height. 

In [ ]:
galton = Table.read_table('galton.csv')
galton

##### We will use the table to see if there is a positive association with parents mid height and child height

In [ ]:
galton.scatter('midparentHeight', 'childHeight')

##### Suppose we want to predict childs height if their parent mid height is 68 inches.  We could find the aveage of childern heights ofparents mid height whos mid height is 68 +/- 0.5 inches tall. 

In [ ]:
galton.scatter('midparentHeight', 'childHeight')
plots.plot([67.5, 67.5], [50, 85], color='red', lw=2)
plots.plot([68.5, 68.5], [50, 85], color='red', lw=2);

In [ ]:
nearby = galton.where('midparentHeight', are.between(67.5, 68.5))
nearby_mean = nearby.column('childHeight').mean()
nearby_mean

In [ ]:
galton.scatter('midparentHeight', 'childHeight')
plots.plot([67.5, 67.5], [50, 85], color='red', lw=2)
plots.plot([68.5, 68.5], [50, 85], color='red', lw=2)
plots.scatter(68, nearby_mean, color='gold', s=50);

##### Lets wrap this calculation into a function.

In [ ]:
def predict(h):
    nearby = galton.where('midparentHeight', are.between(h - 1/2, h + 1/2))
    return nearby.column('childHeight').mean()

In [ ]:
predict(68)

In [ ]:
predict(70)

In [ ]:
predict(73)

##### Now we can apply the function to each mid partent height column to get a prediction of the childs height

In [ ]:
predicted_heights = galton.apply(predict, 'midparentHeight')
predicted_heights

In [ ]:
galton = galton.with_column('predictedHeight', predicted_heights)
galton

In [ ]:
galton.select(
    'midparentHeight', 'childHeight', 'predictedHeight').scatter('midparentHeight')

Galton's calculations and visulaizations were similar to ours (except he had pencil and paper, not Python). He noticed the correlation between parents height and childrens height.  He noticed the averages through the scatter plot was roughly a straight line.  We know this line to be a *regression line* and is one of the most common methods of making predictions.

## Prediction Accuracy
How close is the regression line to the actual child height?  How close the line is to each true value will give us some insight in prediction accuracy and the uncertanity of our model.

Lets create a function to measure the accuacy of the predicted height to the actual height and apply the function to our data.

In [ ]:
def difference(x, y):
    return x - y

In [ ]:
pred_errs = galton.apply(difference, 'predictedHeight', 'childHeight')
pred_errs

In [ ]:
galton = galton.with_column('errors',pred_errs)
galton

We can view a histogram of the errors

In [ ]:
galton.hist('errors')

In [ ]:
galton.hist('errors', group='gender')

## How does the prediction change if we factor gender in the prediction? 

In [ ]:
def predict_gender(h, gender):
    galton_gender = galton.where('gender', are.equal_to(gender))
    nearby = galton_gender.where('midparentHeight', are.between(h - 1/2, h + 1/2))
    return nearby.column('childHeight').mean()

In [ ]:
galton_gender = galton.with_column('predictedHeight', galton.apply(predict_gender, 'midparentHeight', 'gender'))

In [ ]:
galton_gender = galton_gender.with_column('errors', galton_gender.apply(difference, 'predictedHeight', 'childHeight'))

In [ ]:
galton_gender

In [ ]:
galton_gender.select(
    'midparentHeight', 'childHeight', 'predictedHeight').scatter('midparentHeight')

In [ ]:
galton_gender.hist('errors')

In [ ]:
galton_gender.hist('errors', group='gender')

## Grouping

In [ ]:
cones = Table.read_table("cones.csv")

In [ ]:
cones

In [ ]:
cones.group('Flavor')

In [ ]:
cones.drop('Color').group('Flavor', np.average)

In [ ]:
cones.drop('Color').group('Flavor', min)

### Find the total price of the cones by flavor 

In [ ]:
cones.drop('Color').group('Flavor', sum)

## Grouping 2 Columns

In [ ]:
cones.group(['Flavor', 'Color'])

In [ ]:
cones.group(['Flavor', 'Color'], sum)

### Pivot tables rearrage the output of group

In [ ]:
cones.pivot('Flavor', 'Color')

In [ ]:
cones.pivot('Flavor', 'Color', values='Price', collect=sum)

## Load the nba_salaries.csv then relabel the '15-'16 SALARY colum as 'SALARY'

### How much money did each team pay their players?

### What was the average salary at each position?

### What is the max salary of each position on each team?

## Joining

The general format for join is:

<code>table1.join(table1_column_for_joining, table2, table2_column_for_joining)

In [ ]:
cones

In [ ]:
ratings = Table().with_columns('Stars', [4.0, 4.5, 3.5],
                           'Flavor', ["strawberry", "chocolate", "bubblegum"])
ratings

In [ ]:
rated = cones.join('Flavor', ratings, 'Flavor')
rated

## Demo of billionaires

CEO data: 100 = Demoratic party; 200 = Republican Party

In [ ]:
billionaires = Table().read_table('billionaires.csv')
billionaires

In [ ]:
billionaires.group('citizenship').sort(1, descending=True).barh('citizenship')

In [ ]:
billionaires.group('industry').sort(1, descending=True).barh('industry')

In [ ]:
billionaires.pivot('industry', 'citizenship')

In [ ]:
ceos = Table.read_table('ceo.csv')
ceos

In [ ]:
ceos.hist('amount')

In [ ]:
ceos.sort('amount', descending=True)

In [ ]:
ceos.sort('amount', descending=False)

In [ ]:
ceos.hist('amount', bins=np.arange(0, 1e5, 1e3), unit='$1000')

In [ ]:
b_donations = billionaires.join('name', ceos, 'ceo')
b_donations.show()

In [ ]:
b_donations.where('amount', are.below(6e6)).scatter('net worth', 'amount')

In [ ]:
donation_prop = b_donations.column('amount') / b_donations.column('net worth')
b_donations = b_donations.with_column('Contribution%', donation_prop)
b_donations = b_donations.sort('amount', descending=True)
b_donations.set_format('Contribution%', PercentFormatter)
b_donations.show(20)

In [ ]:
b_donations.pivot(['name', 'party'].group(['party'], sum)

In [ ]:
b_donations.pivot('party', 'name', values = 'amount', collect = np.sum).sort('200', descending = True)

In [ ]:
b_donations.pivot('party', 'name', values = 'amount', collect = np.sum).sort('100', descending = True)